In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
!unzip -qq "/content/drive/MyDrive/Colab Notebooks/statoil-iceberg-classifier-challenge.zip"

In [ ]:
!pip install py7zr

In [4]:
import os 
import py7zr
import pandas as pd

In [ ]:
with py7zr.SevenZipFile("/content/train.json.7z", 'r') as archive:
    archive.extractall(path="/content/train")
with py7zr.SevenZipFile("/content/test.json.7z", 'r') as archive:
    archive.extractall(path="/content/test")
with py7zr.SevenZipFile("/content/sample_submission.csv.7z", 'r') as archive:
    archive.extractall(path="/content/sample_submission")
for dirname, _, filenames in os.walk('/content'): 
    for filename in filenames: 
        print(os.path.join(dirname, filename))

In [6]:
train=pd.read_json("/content/train/data/processed/train.json")
test=pd.read_json("/content/test/data/processed/test.json")
sample_submission=pd.read_csv("/content/sample_submission/data/processed/sample_submission.csv")

- 파일 불러오기 까지 19/100 줄

In [7]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [9]:
for i in range(0,1604):
    if train.iloc[i, 3]=='na':
        train.iloc[i, 3]=0

In [10]:
train['inc_angle']=train['inc_angle'].astype('float')
train['id']=le.fit_transform(train['id'])
test['id']=le.fit_transform(test['id'])

In [12]:
import numpy as np
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis], X_band_test_2[:, :, :, np.newaxis], ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)

In [14]:
Y_train=np.array(train['is_iceberg'])
print(X_train.shape, X_test.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


- 데이터 전처리까지 36/100 줄

In [42]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from tensorflow.keras.optimizers import SGD, RMSprop
from keras import optimizers
import tensorflow as tf

In [43]:
np.random.seed(0)
tf.random.set_seed(0)

In [44]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=0)

In [45]:
y_train=y_train.reshape(-1,1)
y_val=y_val.reshape(-1,1)

In [46]:
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape, X_test.shape)

(1443, 75, 75, 3) (1443, 1) (161, 75, 75, 3) (161, 1) (8424, 75, 75, 3)


- 모델 적용 전까지 49/100줄

VGG19 모델 Summary 참고하여 모델 쌓기 

In [ ]:
model = Sequential()
model.add(Conv2D(input_shape=(75,75,3),filters=32,kernel_size=(3,3),padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=32,kernel_size=(3,3),padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.3))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.4))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.5))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.6))
model.add(Flatten())
model.add(Dense(units=512,activation="relu"))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))
opt=SGD(lr=0.001, momentum=0.9)
model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=1e-4), metrics=['acc'])

- 98/100줄

In [49]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_113 (Conv2D)          (None, 75, 75, 32)        896       
_________________________________________________________________
batch_normalization_119 (Bat (None, 75, 75, 32)        128       
_________________________________________________________________
conv2d_114 (Conv2D)          (None, 75, 75, 32)        9248      
_________________________________________________________________
batch_normalization_120 (Bat (None, 75, 75, 32)        128       
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 37, 37, 32)        0         
_________________________________________________________________
dropout_35 (Dropout)         (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_115 (Conv2D)          (None, 37, 37, 64)       

In [48]:
history=model.fit(x_train, y_train, batch_size=10, epochs=100, validation_data=(x_val,y_val))

Epoch 1/100
145/145 [==============================] - 8s 30ms/step - loss: 0.7863 - acc: 0.5870 - val_loss: 0.7968 - val_acc: 0.4969
Epoch 2/100
145/145 [==============================] - 3s 20ms/step - loss: 0.7126 - acc: 0.6334 - val_loss: 1.3408 - val_acc: 0.4969
Epoch 3/100
145/145 [==============================] - 3s 20ms/step - loss: 0.6987 - acc: 0.6320 - val_loss: 0.7484 - val_acc: 0.5466
Epoch 4/100
145/145 [==============================] - 3s 20ms/step - loss: 0.6732 - acc: 0.6480 - val_loss: 0.8373 - val_acc: 0.5652
Epoch 5/100
145/145 [==============================] - 3s 20ms/step - loss: 0.6745 - acc: 0.6452 - val_loss: 0.7657 - val_acc: 0.5528
Epoch 6/100
145/145 [==============================] - 3s 20ms/step - loss: 0.6367 - acc: 0.6798 - val_loss: 0.5615 - val_acc: 0.7205
Epoch 7/100
145/145 [==============================] - 3s 20ms/step - loss: 0.6168 - acc: 0.6771 - val_loss: 0.5841 - val_acc: 0.6957
Epoch 8/100
145/145 [==============================] - 3s 20ms

- 100/100 줄 이므로 종료